In [1]:
from data.dataset import NERDataset
from models.networks import GlobalContextualDeepTransition
from tqdm.notebook import tqdm
import torch

In [2]:
sourceName = 'data/conll03/eng.train.src'
targetName = 'data/conll03/eng.train.trg'
data = NERDataset(sourceName, targetName, 256)
loader = data.getLoader(4096)

In [3]:
256 * 256

65536

In [4]:
max(data.wordIdx.values()), len(data.wordIdx)

(23624, 23625)

In [5]:
numChars = 100
charEmbedding = 128
numWords = len(data.wordIdx)
wordEmbedding = 300
contextOutputUnits = 128
contextTransitionNumber = transitionNumber = 4
encoderUnits = 256
decoderUnits = 256

In [6]:
model = GlobalContextualDeepTransition(numChars, charEmbedding, numWords,
                     wordEmbedding, contextOutputUnits, contextTransitionNumber,
                        encoderUnits, decoderUnits, transitionNumber)
numParams = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {numParams:,}")
# 6,774,656

Trainable parameters: 6,774,656


In [7]:
# x = model.parameters()
# y = model.state_dict().items()

# for (key, value), par in zip(y, x):
#     if par.requires_grad:
#         print(par.numel())

12800
49152
128
71168
71168
71168
54784
54784
16384
128
128
128
128
128
128
16384
16384
16384
128
128
128
128
16384
16384
16384
128
128
128
128
16384
16384
16384
128
128
128
128
16384
16384
16384
128
128
128
128
71168
71168
71168
54784
54784
16384
128
128
128
128
128
128
16384
16384
16384
128
128
128
128
16384
16384
16384
128
128
128
128
16384
16384
16384
128
128
128
128
16384
16384
16384
128
128
128
128
240640
240640
240640
175104
175104
65536
256
256
256
256
256
256
65536
65536
65536
256
256
256
256
65536
65536
65536
256
256
256
256
65536
65536
65536
256
256
256
256
65536
65536
65536
256
256
256
256
240640
240640
240640
175104
175104
65536
256
256
256
256
256
256
65536
65536
65536
256
256
256
256
65536
65536
65536
256
256
256
256
65536
65536
65536
256
256
256
256
65536
65536
65536
256
256
256
256
196608
196608
196608
131072
131072
65536
256
256
256
256
256
256
65536
65536
65536
256
256
256
256
65536
65536
65536
256
256
256
256
65536
65536
65536
256
256
256
256
65536
65536
65536
256
2

In [9]:
model = model.cuda()
tot = 0
for batch in tqdm(loader):
    words, chars, mask, targets = batch
    batch = words.cuda(), chars.cuda(), mask.cuda(), targets.cuda()
    loss = model.training_step(batch, 0)
    tot += loss.item()